In [ ]:
from dotenv import load_dotenv

load_dotenv("../../../config/.env")

True

In [4]:
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

model = init_chat_model("deepseek-chat", model_provider="deepseek")
search = TavilySearch(max_results=2)
tools = [search]
memory = MemorySaver()
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [5]:
config = {"configurable": {"thread_id": "thread_1"}}
input_message = {
    "role": "user",
    "content": "你好，我叫Frank，我在北京。",
}

for step in agent_executor.stream(
    {"messages": [input_message]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

你好，我叫Frank，我在北京。
================================== Ai Message ==================================

你好Frank！很高兴认识你。北京是个很棒的城市，有着丰富的历史文化和现代化的发展。

有什么我可以帮助你的吗？无论是关于北京的信息、旅行建议、文化探索，还是其他任何问题，我都很乐意为你提供帮助。


In [6]:
input_message = {
    "role": "user",
    "content": "我所在的地方的天气是怎样的？",
}

for step in agent_executor.stream(
    {"messages": [input_message]},
    config,
    stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

我所在的地方的天气是怎样的？
================================== Ai Message ==================================

我来帮你查询一下北京当前的天气情况。
Tool Calls:
  tavily_search (call_00_gt3eqR8btnrTjSWG43wNLnoZ)
 Call ID: call_00_gt3eqR8btnrTjSWG43wNLnoZ
  Args:
    query: 北京天气 实时天气 今天
    search_depth: basic
    time_range: day
================================= Tool Message =================================
Name: tavily_search

{"query": "北京天气 实时天气 今天", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://tianqi.moji.com/today/china/beijing/beijing", "title": "今天北京市天气_今日天气预报- 墨迹天气", "content": "北京市今天实况：25度阴，湿度：55%，西南风：1级。白天：27度,多云。 夜间：小雨，17度，天气较热，墨迹天气建议您选择短袖上衣加七分裤的搭配，针织衫是进出空调房的必备单品。", "score": 0.8055255, "raw_content": null}, {"url": "https://aqicn.org/city/beijing/cn/m/", "title": "北京空气污染：实时PM2.5空气质量指数（AQI） - Beijing Air Quality.", "content": "| 61 | Beijing (北京) 今天08:00 - 1 小时前 | | 65 | Dongcheng Don

In [7]:
search_result = search.invoke("北京明天的天气怎么样？")

print(search_result)

{'query': '北京明天的天气怎么样？', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.weather.com.cn/weather/101010100.shtml', 'title': '预报- 北京', 'content': '22日（明天）. 阴转多云. 24℃/17℃. <3级. 23日（后天）.', 'score': 0.77548623, 'raw_content': None}, {'url': 'https://tianqi.so.com/weather/', 'title': '【北京天气预报】北京天气预报7天,10天,15天_全国天气网', 'content': '今日天气 明日天气 市气象台2025年09月15日22时20分发布大风蓝色预警信号：受冷空气影响，预计9月16日05时至20时，我市大部分地区有3、4级偏北风，阵风6、7级，山区局地阵风可达8级以上，请注意防范。 市气象台2025年09月15日22时20分发布大风蓝色预警信号：受冷空气影响，预计9月16日05时至20时，我市大部分地区有3、4级偏北风，阵风6、7级，山区局地阵风可达8级以上，请注意防范。 ### 当前天气信息 天气：阴 24℃ 体感：24℃ 风向：北风 风力：3级 气压：1011 湿度：34% 日出：05:55 日落：18:23 ### 空气质量 空气很棒，快出门呼吸新鲜空气吧。 无需戴口罩 适宜外出 适宜开窗 关闭净化器 ### 主要污染物 PM2.5 4优 PM10 11优 O3 56优 9优 3优 08时 11时 14时 17时 20时 23时 温度：13~26℃ | 南宁 | 深圳 | 承德 | 吐鲁番 | | 无锡 | 沈阳 | 昆明 | 石家庄 | | 南宁 | 福州 | 长沙 | 天津 | | 昆明 | 贵阳 | 沈阳 | 北京 | | 重庆 | 西宁 | 澳门 | 杭州 | | 合作 | 抚顺 | 临夏 | 南平 | | 无锡 | 集宁 | 南平 | 宜春 | | 重庆 | 淮北 | 孝感 | 阳泉 | | 石河子 | 北京 | 酒泉 | 泰安 |', 'score': 0.7159213, 'raw_cont

In [8]:
for step, metadata in agent_executor.stream(
    {"messages": [input_message]},
    config,
    stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

我来|为你|查询|北京|当前的|天气|情况|。|让我|再|查询|更|详细的|北京|实时|天气|信息|。|根据|最新的|天气|信息|，|北京|当前的|天气|情况|如下|：

|**|🌤|️| |今日|天气|概况|：|**
|-| **|当前|温度|**|：|约|25|-|26|°|C|
|-| **|湿度|**|：|约|45|-|85|%（|不同|时段|变化|较大|）
|-| **|风向|风力|**|：|西南|风|或|南|风|，|微风|（|<|3|级|）
|-| **|天气|状况|**|：|多云|到|阴|天|

|**|📅| |今日|详细|预报|：|**
|-| **|最高|温度|**|：|26|°|C|
|-| **|最低|温度|**|：|17|°|C|
|-| **|天气|变化|**|：|阴|天|转|小雨|
|-| **|日出|时间|**|：|06|:|02|
|-| **|日落|时间|**|：|18|:|11|

|**|👕| |穿衣|建议|：|**
|今天|天气|比较|温暖|，|建议|穿着|：
|-| |短|袖|或|薄|长|袖|上衣|
|-| |长|裤|或|七|分|裤|
|-| |带|一件|薄|外套|或|针织|衫|，|方便|进出|空调|房|
|-| |晚上|有小|雨|，|建议|携带|雨|具|

|**|💡| |温馨提示|：|**|
|-| |湿度|变化|较大|，|注意|适时|增减|衣物|
|-| |晚上|有小|雨|，|出行|请|带|伞|
|-| |空气质量|中等|，|敏感|人群|建议|减少|户外|活动|时间|

|整体|来说|，|今天|北京的|天气|还是比较|舒适的|，|适合|户外|活动|！|